## 现在进行达成率的计算 (所有APP维度) 简化版

## 进行第一轮筛选并且清洗数据

选出
* IAA独代+gdt+APP
* IAA独代+mp+APP
* 非IAA独代+gdt+APP
* 非IAA独代+mp+APP

共4个维度.


**注意我们这里区分APP 和小游戏维度，因为判断二者是否达成的逻辑不同**

In [1]:
# 进行筛选 1

import openpyxl
import pandas as pd
import numpy as np

df=pd.read_excel('2部_游戏行业报表-OPS 0828-0903 2.xlsx',sheet_name = 'Sheet1')  #读取初始数据 excel表格数据

# define a function to determine whether it is IAA独代 or others
    

dfa= df.loc[df['游戏集团'].isin(['IAA游戏独代内投','腾讯科技(深圳)有限公司']) ]  # 筛选出IAA独代 商品
dfb= df[(df['游戏集团']!='IAA游戏独代内投') &  (df['游戏集团']!='腾讯科技(深圳)有限公司')] # 筛选出 非IAA独代商品

df1=dfa[(dfa['是否微广流量']=='gdt')]  # 进行筛选 IAA独代+gdt 的商品 并保存为df1 dataframe 格式文件
df2=dfa[(dfa['是否微广流量']=='mp')]  # 进行筛选  IAA独代+mp 的商品 并保存为df2 dataframe 格式文件
df3=dfb[(dfb['是否微广流量']=='gdt')]  # 进行筛选 非IAA独代+gdt的商品 并保存为df3 dataframe 格式文件
df4=dfb[(dfb['是否微广流量']=='mp')]  # 进行筛选 非IAA独代+ mp 的商品 并保存为df4 dataframe 格式文件

# Maybe we could use a for loop here to simplify the process
list=[df1,df2,df3,df4]
list1=[] # 创建一个空的list 用于保存筛选过后为APP商品 以及 4个维度下的 dataframe 

j=0
while j<4:
    df=list[j]
    list1.append(df.loc[df['商品类型'].isin([12,19])])  # 用于保存筛选过后的 为APP的商品 & 注意 12,19 为APP 类型
    j+=1
            
# 进行数据清洗 去掉消耗为0 和 ROI出价为~ 的值 & 使用while loop 来简化清洗过程
list2=[]  #创建一个空的list2 用于保存清洗过后为APP商品 以及 4个维度下的 dataframe
list3=[]  #创建一个空的list3 用于保存筛选不合格为APP商品 以及 4个维度下的 dataframe 用于检查

n=0
while n<4:
        df=list1[n]
        list2.append(df[(df['首日变现ROI广告消耗(单位元)(元)(当前值)']!=0)& (df['ROI出价(点击口径)(旧)(元)(当前值)']!='~')& (df['游戏集团']!='0') ])  # 进行筛选掉竞价消耗为0的商品与筛选掉ROI出价为~ & 游戏集团 为0 的商品 
        list3.append(df[(df['首日变现ROI广告消耗(单位元)(元)(当前值)']==0)| (df['ROI出价(点击口径)(旧)(元)(当前值)']=='~') ])  # 筛选出竞价消耗为0的商品和ROI出价为~的商品 用于检查           
        n+=1

# define save to simiplify the process    
def save(df1, df2, writer1,write2, name1='A'): # 默认sheet_name 参数 为 A & writer1,writer2 为 将要写入的 excel 文件名
        df1.to_excel(writer1, sheet_name= name1,index = False) # 保存原文件 为 sheet 1 且加参数而不加索引
        df2.to_excel(writer2, sheet_name= name1,index = False) # 保存 df2 为 sheet 2 文件 且加参数而不加索引
 
list4=['IAA独代+gdt+APP 1','IAA独代+mp+APP 1','非IAA独代+gdt+APP 1','非IAA独代+mp+APP 1']

with pd.ExcelWriter('Stat1.xlsx') as writer1:
    with pd.ExcelWriter('Bad Cases.xlsx') as writer2:
        for k in range(0,4):
            save(list2[k],list3[k],writer1,writer2,list4[k])
        
    
print('筛选与清洗成功！')


筛选与清洗成功！


##  第一次统计在各个维度下的总账户数量和总竞价消耗以及达成账户和达成消耗

In [2]:
# 进行统计 1

import openpyxl
import pandas as pd
import numpy as np

list0=['IAA独代+gdt+APP 1','IAA独代+mp+APP 1','非IAA独代+gdt+APP 1','非IAA独代+mp+APP 1']
list=[]
for n in range(0,4):
    list.append(pd.read_excel('Stat1.xlsx',sheet_name = list0[n]))


# 批量创建新的dataframe 以便后期写入数据
df = pd.DataFrame() # 创建自定义列名称的DataFrame & 这里df 放在循环外 防止被覆盖
list1=[df,df,df,df] # 在loop 外声明确保list1 里元素不丢失

for j in range(0,4): # create a for loop to simplify the process 
    list1[j]=list[j].loc[:,["翻译后(如有).2",'首日变现ROI广告消耗(单位元)(元)(当前值)']]#！！！ 翻译后(如有).2 为原表中 游戏名 请核对好该细节
    list1[j].rename(columns={"翻译后(如有).2": '游戏名', '首日变现ROI广告消耗(单位元)(元)(当前值)':'总账户数/竞价消耗(元)'},inplace = True)
    #print(list1[j])
    

# 透视表 统计总账户数和总竞价消耗 & 这里使用循环与自定义函数简化计算
list2=[] # 创建一个空list 用于存储 各个透视表
for i in range(0,4):
    list2.append(pd.pivot_table(list1[i],index=["游戏名"], \
                       aggfunc=['count','sum'],margins=True)) # 这里为统计过后数据
 


# 保存各个dataframe 在 'Stat 1.xlsx'  excel 文件里

list3=['IAA独代+gdt+APP 1','IAA独代+gdt+APP 2','IAA独代+mp+APP 1','IAA独代+mp+APP 2','非IAA独代+gdt+APP 1','非IAA独代+gdt+APP 2','非IAA独代+mp+APP 1','非IAA独代+mp+APP 2']

# 定义save 函数简化计算 
def save(df1, df2, writer,name1='A', name2='B'): # 默认sheet_name 参数 为 A,B & writer 为 将要写入的 excel 文件名
        df1.to_excel(writer, sheet_name= name1,index = False) # 保存原文件 为 sheet 1 且加参数而不加索引
        df2.to_excel(writer, sheet_name= name2) # 保存 df2 为 sheet 2 文件 且加参数而不加索引

with pd.ExcelWriter('Stat1.xlsx') as writer:
    for k in range(0,4):
        save(list[k],list2[k],writer, list3[2*k],list3[2*k+1]) # 注意 list 为原文件, list2 为统计完的数据, list3 为表名
         
print('统计成功！')  

统计成功！


In [3]:
# 进行统计 2.1, 统计各个APP 维度下的 首日实际 ROI 商品

import openpyxl
import pandas as pd
import numpy as np
from openpyxl import load_workbook # import load_workbook to accesss excel data

# 读取excel 数据
wb = load_workbook('Stat1.xlsx')

list=['IAA独代+gdt+APP 1','IAA独代+mp+APP 1','非IAA独代+gdt+APP 1','非IAA独代+mp+APP 1']
list1=[] # create a new list to store each work_sheet

for i in range(0,4):
    list1.append(wb[list[i]]) # 用 df.append.()写入数据

# 创建新的列 "首日实际ROI"
for j in range(0,4):
    list1[j]['Q1']="首日实际ROI"
   
#  use 4 'for' loops to compute '首日实际ROI'

for j in range(0,4):
    for i in range(2,list1[j].max_row+1): # here we add 1 because range will not include ws.max_row in Python 
         list1[j].cell(i, 17).value = list1[j].cell(i, 15).value / (list1[j].cell(i,11).value *10000) # 单元格计算用 ws1.cell(row,column).value

wb.save("Stat1.xlsx")
print('统计成功！')  

统计成功！


In [4]:
# 进行统计 2.2, 统计各个APP 维度下的 达成 & 未达成 商品

import openpyxl
import pandas as pd
import numpy as np
from openpyxl import load_workbook # import load_workbook to accesss excel data

def compare(a, b): 
        if  a > b:   # a 指ROI出价；b指实际ROI
         return '未达成'
        else: 
         return '达成'

# 以dataframe 的形式 读取excel 数据 
list=['IAA独代+gdt+APP 1','IAA独代+gdt+APP 2','IAA独代+mp+APP 1','IAA独代+mp+APP 2','非IAA独代+gdt+APP 1','非IAA独代+gdt+APP 2','非IAA独代+mp+APP 1','非IAA独代+mp+APP 2']
list1=[]  # 用于 储存 预处理数据 or 代号以1 结尾的数据
list2=[]  # 用于 储存 已统计数据 or 代号以2 结尾的数据

# for loop 读取 代号以1 结尾的数据
for i in range(0,4):
    list1.append(pd.read_excel('Stat1.xlsx',sheet_name = list[2*i]))
    
 
#  for loop 读取 代号以 2 结尾的数据
for j in range(0,4):
    list2.append(pd.read_excel('Stat1.xlsx',sheet_name = list[2*j+1]))

    
#change the type of this column from object to float
for k in range(0,4):
    list1[k]['ROI出价(点击口径)(旧)(元)(当前值)']= pd.to_numeric(list1[k]['ROI出价(点击口径)(旧)(元)(当前值)'], errors='coerce') # change the type of this column from object to float


# 用compare 判断是否达成 & 在此之前 还需要一个函数 用于判断 是否有 Sheet 存在空值情况
for k in range(0,4):
    df = list1[k]
    if df['首日实际ROI'].isnull().all() : # == np.nan or .all() 判断某列是否全部为空值
        size = df.index.size   # 行数
        column = df.shape[1] # 列数
        x=np.arange(column) # 注意这里的 np.arange()  只有 一个 r 在 arange 里
        x[::-1].sort()  # 将 数组按从大到小排列, 从column-1 至 0
        list1[k].loc[size]= x
        print(list[2*k]+ ' Sheet 中存在空值！','\n')
    else:
        print(list[2*k]+ ' Sheet 中没有空值！','\n')

for k in range(0,4):
    list1[k]['是否达成'] = list1[k].apply(lambda x: compare(x['ROI出价(点击口径)(旧)(元)(当前值)'], x['首日实际ROI']), axis = 1) # lambda x 这里为简易函数

# 定义 save2 函数简化 写入过程
def save2(df1, df2, writer,name1='A', name2='B'): # 默认sheet_name 参数 为 A,B & writer 为 将要写入的 excel 文件名
        df1.to_excel(writer, sheet_name= name1,index = False) # 保存原文件 为 sheet 1 且加参数而不加索引
        df2.to_excel(writer, sheet_name= name2,index = False) # 保存 df2 为 sheet 2 文件 且加参数而不加索引


with pd.ExcelWriter('Stat2.xlsx') as writer:
    for n in range(0,4):
        save2(list1[n],list2[n],writer, list[2*n],list[2*n+1]) # 注意 list 2 为统计完的文件, list1 为此cell处理完的数据, list 为表名
         
print('统计成功！')       
        

IAA独代+gdt+APP 1 Sheet 中没有空值！ 

IAA独代+mp+APP 1 Sheet 中没有空值！ 

非IAA独代+gdt+APP 1 Sheet 中存在空值！ 

非IAA独代+mp+APP 1 Sheet 中存在空值！ 

统计成功！


In [5]:
#进行统计 3, 统计各个维度下 的 达成 & 未达成 商品 的账户数和总竞价消耗 并且将计算完的数据保存在Stat2

import openpyxl
import pandas as pd
import numpy as np
from openpyxl import load_workbook # import load_workbook to accesss excel data

# 以dataframe 的形式 读取excel Stat2 数据 

list=['IAA独代+gdt+APP 1','IAA独代+gdt+APP 2','IAA独代+mp+APP 1','IAA独代+mp+APP 2','非IAA独代+gdt+APP 1','非IAA独代+gdt+APP 2','非IAA独代+mp+APP 1','非IAA独代+mp+APP 2']
list1=[]  # 用于 储存 预处理数据 or 代号以1 结尾的数据
list2=[]  # 用于 储存 已统计数据 or 代号以2 结尾的数据

# for loop 读取 代号以1 结尾的数据
for i in range(0,4):
    list1.append(pd.read_excel('Stat2.xlsx',sheet_name = list[2*i]))
     
# for loop 读取 代号以 2 结尾的数据
for j in range(0,4):
    list2.append(pd.read_excel('Stat2.xlsx',sheet_name = list[2*j+1]))

# 进行最后的筛选统计
list3=[]   
n=0
while n<4:
        df=list1[n]
        list3.append(df[df['是否达成']=="达成"]) # 进行筛选出 达成 的项 并保存为dfa dataframe 格式文件
        n+=1

# 批量创建新的dataframe 以便后期写入数据
df = pd.DataFrame() # 创建自定义列名称的DataFrame & 这里df 放在循环外 防止被覆盖
list4=[df,df,df,df] # 在loop 外声明确保list4 里元素不丢失

for j in range(0,4): # create a for loop to simplify the process
    list4[j]=list3[j].loc[:,["翻译后(如有).2",'首日变现ROI广告消耗(单位元)(元)(当前值)']] #！！！ 翻译后(如有).2 为原表中 游戏名 请核对好该细节
    list4[j].rename(columns={"翻译后(如有).2": '游戏名', '首日变现ROI广告消耗(单位元)(元)(当前值)': '达成总账户数/达成竞价消耗(元)'},inplace = True)


# pd.pivot_table 透视表进行统计 & 统计总账户数和总竞价消耗 & 这里使用循环与自定义函数简化计算
list5=[] # 创建一个空list 用于存储 各个透视表
for i in range(0,4):
    list5.append(pd.pivot_table(list4[i],index=["游戏名"], \
                       aggfunc=['count','sum'],margins=True)) # 这里为统计过后数据


# 定义 save3 函数简化 写入过程
def save3(df1, df2,df3, writer,name1='A', name2='B',name3='C'): # df1,2,3 为dataframe 数据 & 默认sheet_name 参数 为 A,B,C & writer 为 将要写入的 excel 文件名 
        df1.to_excel(writer, sheet_name= name1,index = False) # 保存原文件 为 sheet 1 且加参数而不加索引
        df2.to_excel(writer, sheet_name= name2,index = False) # 保存 df2 为 sheet 2 文件 且加参数而不加索引
        df3.to_excel(writer, sheet_name= name3,) # 保存 df3为 sheet 3 文件 

# 再创建一个list6 用于读取和写入 表名
list6=['IAA独代+gdt+APP 3','IAA独代+mp+APP 3','非IAA独代+gdt+APP 3','非IAA独代+mp+APP 3']

with pd.ExcelWriter('Stat2.xlsx') as writer:
    for n in range(0,4):
        save3(list1[n],list2[n],list5[n], writer, list[2*n],list[2*n+1],list6[n]) # 注意 list3 为 此次cell 处理数据; list 2 之前统计完的文件; list1 为原数据;  list, list6 为表名 
         
print('统计成功！')           
    


统计成功！


## 最后我们进行达成率的计算

In [6]:
import openpyxl
import pandas as pd
import numpy as np
from openpyxl import load_workbook # import load_workbook to accesss excel data

wb = load_workbook('达成率表格.xlsx')
wb1 = load_workbook('Stat2.xlsx')

ws = wb['Sheet1']

# 读取excel 数据

list=['IAA独代+gdt+APP 2','IAA独代+mp+APP 2','非IAA独代+gdt+APP 2','非IAA独代+mp+APP 2']
list1=['IAA独代+gdt+APP 3','IAA独代+mp+APP 3','非IAA独代+gdt+APP 3','非IAA独代+mp+APP 3']
list2=[] # create a new list to store each work_sheet 2
list3=[] # create a new list to store each work_sheet 3

for k in range(0,4):
    list2.append(wb1[list[k]])
    list3.append(wb1[list1[k]])
    
   
 #  use 'for' loops to compute ' 达成率 & 这里 用 max(value,1) 和 max(value,0.0000001) 为了避免公式出现 /0 的情况
                 
for j in range(0,4):
    ws.cell(2*j+2,4).value = list3[j].cell(list3[j].max_row,2).value /  max(list2[j].cell(list2[j].max_row,2).value,1) # 将 达成率(账户数) 写在 D2,4,6,8 上 & D2 刚好 是 IAA独代+ gdt + APP...
    ws.cell(2*j+2,5).value= list3[j].cell(list3[j].max_row,3).value /  max(list2[j].cell(list2[j].max_row,3).value,0.00000001) # 将 达成率(消耗数) 写在 E2,4,6,8 上 & E2 刚好 是 IAA独代+ gdt + APP...
    print(ws.cell(2*j+2,4).value,ws.cell(2*j+2,5).value,'\n' )
                 

wb.save("Result.xlsx")

print('计算成功！')    
 

0.8914027149321267 0.9903479019199811 

0.7835051546391752 0.9738025775254997 

0.0 0.0 

0.0 0.0 

计算成功！
